<a href="https://colab.research.google.com/github/JimmyCh2001/kkcompany_datagame_schubert/blob/068_branch/kkcompany_datagame_schubert_019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Get Data from Kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c datagame-2023

 94% 379M/401M [00:02<00:00, 169MB/s]
100% 401M/401M [00:02<00:00, 157MB/s]


In [ ]:
!unzip datagame-2023

Archive:  datagame-2023.zip
  inflating: label_test_source.parquet  
  inflating: label_train_source.parquet  
  inflating: label_train_target.parquet  
  inflating: meta_song.parquet       
  inflating: meta_song_composer.parquet  
  inflating: meta_song_genre.parquet  
  inflating: meta_song_lyricist.parquet  
  inflating: meta_song_producer.parquet  
  inflating: meta_song_titletext.parquet  
  inflating: sample.csv              


#Import Library

In [ ]:
import pandas as pd
import numpy as np

#Data Preprocessing

Load data from parquet file to code

In [36]:
label_test = pd.read_parquet("/content/label_test_source.parquet")
label_train = pd.read_parquet("/content/label_train_source.parquet")
label_target = pd.read_parquet("/content/label_train_target.parquet")
meta_song = pd.read_parquet("/content/meta_song.parquet")
meta_song_composer = pd.read_parquet("/content/meta_song_composer.parquet")
meta_song_genre = pd.read_parquet("/content/meta_song_genre.parquet")
meta_song_lyricist = pd.read_parquet("/content/meta_song_lyricist.parquet")
meta_song_producer = pd.read_parquet("/content/meta_song_producer.parquet")
meta_song_title = pd.read_parquet("/content/meta_song_titletext.parquet")

In [37]:
meta_song

,song_id,artist_id,song_length,album_id,language_id,album_month
0,10a46165bb84e056438e06c35763ee61,39.0,202.0,202.0,3.0,2000-01
1,ff025522d0f8e7198a75a4e03edce55c,133.0,278.0,9615.0,3.0,1995-01
2,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06
3,ed8b14c6d0fccd1a95f9140933f809f3,114.0,263.0,21059.0,3.0,1989-01
4,2ce5e96e0213a6dac83c54b003418726,112.0,194.0,22531.0,3.0,2003-01
...,...,...,...,...,...,...
1030707,2e317504e175c1c6d18fa47b21b1a7f5,NaN,NaN,NaN,NaN,None
1030708,60f94d53d47ff60130924a320345b8f1,NaN,NaN,NaN,NaN,None
1030709,2c7c7dd5713f93222429a5f05732de3b,NaN,NaN,NaN,NaN,None
1030710,04e7a4fa2804ff363cf985fb83ac5d82,NaN,NaN,NaN,NaN,None


Merge Meta Data to one variabe

In [43]:
list_meta = [meta_song_composer, meta_song_genre, meta_song_title, meta_song_lyricist, meta_song_producer]

for meta in list_meta:
  meta_song = pd.merge(meta_song, meta.drop_duplicates(subset='song_id'), on="song_id", how="left")

meta_song

,song_id,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,10a46165bb84e056438e06c35763ee61,39.0,202.0,202.0,3.0,2000-01,NaN,ce4db56f6a48426643b08038139a8a75,b2153fe5a86fd746903746a219d40083,NaN,95584be121b41e3e758bd99e9cc07abd
1,ff025522d0f8e7198a75a4e03edce55c,133.0,278.0,9615.0,3.0,1995-01,47580d357c6779d6a244c514b0acdc72,ce4db56f6a48426643b08038139a8a75,c1079ef109db2aba72f78c632ab73803,NaN,8e9704c1d8cf460fe5a2eb4470e77d3e
2,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06,3211cf51bfa3afff4f264a110212f615,ce4db56f6a48426643b08038139a8a75,c1079ef109db2aba72f78c632ab73803,3211cf51bfa3afff4f264a110212f615,086136ec34f3ff6eb89cfa7c446bd69f
3,ed8b14c6d0fccd1a95f9140933f809f3,114.0,263.0,21059.0,3.0,1989-01,a3ea818b4ee4f8fb4bf87faef36eab79,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,NaN,652bf7f3d54c00c3f9b3d732e8c6dbd1
4,2ce5e96e0213a6dac83c54b003418726,112.0,194.0,22531.0,3.0,2003-01,9120390d224ed7a652a879c8538edb94,ce4db56f6a48426643b08038139a8a75,c1079ef109db2aba72f78c632ab73803,NaN,d57746971ee7f65f98bf8cbe8f070978
...,...,...,...,...,...,...,...,...,...,...,...
1030707,2e317504e175c1c6d18fa47b21b1a7f5,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1030708,60f94d53d47ff60130924a320345b8f1,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1030709,2c7c7dd5713f93222429a5f05732de3b,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1030710,04e7a4fa2804ff363cf985fb83ac5d82,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


Check Data Type of each columns in **meta_song**

In [44]:
column_data_types = meta_song.dtypes

print("Data Types of Each Column:")
print(column_data_types)

Data Types of Each Column:
song_id           object
artist_id        float64
song_length      float64
album_id         float64
language_id      float64
album_month       object
composer_id       object
genre_id          object
title_text_id     object
lyricist_id       object
producer_id       object
dtype: object


In [45]:
column_data_type_train = label_train.dtypes

print("Data Types of Each Column:")
print(column_data_type_train)

Data Types of Each Column:
session_id          int64
song_id            object
unix_played_at      int64
play_status         int64
login_type          int64
listening_order     int64
dtype: object


In [46]:
#Drop those record that had the play_status = -1
label_train_dropped = label_train.replace(-1, np.nan).dropna()
label_test_dropped = label_test.replace(-1, np.nan).dropna()

label_test_dropped

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,598,43519381e8351d7c6afcc66bcc0a0636,1660177283,1.0,7,1
1,598,928e2e8e316f29efd986fa8d23429431,1660177503,1.0,7,2
2,598,5229859624d3b58c35646154cf366bf4,1660177582,1.0,7,3
3,598,49e50187f7a7f658623807780f499a15,1660177584,1.0,7,4
4,598,bc8b660fe4d4bfa0a13334beae0cb750,1660177587,1.0,7,5
...,...,...,...,...,...,...
2861275,714498,13d62012feb791ded330b17098230e0d,1666147178,0.0,7,16
2861276,714498,a3f4cb9a06f48f2e9ea381be3afc64b3,1666147262,0.0,7,17
2861277,714498,9bb5e3b8b43fe1e306459c89ab263823,1666147366,0.0,7,18
2861278,714498,e1ec04cca3d78e9e7fd9793ae45d3638,1666147464,0.0,7,19


In [52]:
label_train_dropped

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,751,6027767fad949f3ca5e772df04924949,1659598909,1.0,7,1
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1.0,7,2
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0.0,7,3
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0.0,7,4
4,751,8b32f88104ecf859be934d9b45f30cd1,1659599218,1.0,7,5
...,...,...,...,...,...,...
11445175,458622,3ba8c1e455e27172edec64deff3a8b15,1662339887,2.0,7,16
11445176,458622,3ba8c1e455e27172edec64deff3a8b15,1662340090,2.0,7,17
11445177,458622,3ba8c1e455e27172edec64deff3a8b15,1662340294,2.0,7,18
11445178,458622,3ba8c1e455e27172edec64deff3a8b15,1662340497,2.0,7,19


#Merge the train data set with meta data to include more information to train

In [48]:
train_data = pd.merge(label_train_dropped, meta_song, on='song_id', how='left')
train_data

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,751,6027767fad949f3ca5e772df04924949,1659598909,1.0,7,1,10471734.0,262.0,58342193.0,3.0,2019-07,877e9712bc464206428b5108cfe8b13a,ce4db56f6a48426643b08038139a8a75,63317dd11041a8e97b54e28d6957eb67,877e9712bc464206428b5108cfe8b13a,NaN
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1.0,7,2,29341904.0,110.0,NaN,NaN,None,482548faa68907f9a5ba92d0bfa1c3cf,NaN,c1079ef109db2aba72f78c632ab73803,482548faa68907f9a5ba92d0bfa1c3cf,d28dcd6115aba63c33d97abbc5746184
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0.0,7,3,29341904.0,110.0,NaN,NaN,None,482548faa68907f9a5ba92d0bfa1c3cf,NaN,c1079ef109db2aba72f78c632ab73803,482548faa68907f9a5ba92d0bfa1c3cf,d28dcd6115aba63c33d97abbc5746184
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0.0,7,4,29341904.0,110.0,NaN,NaN,None,482548faa68907f9a5ba92d0bfa1c3cf,NaN,c1079ef109db2aba72f78c632ab73803,482548faa68907f9a5ba92d0bfa1c3cf,d28dcd6115aba63c33d97abbc5746184
4,751,8b32f88104ecf859be934d9b45f30cd1,1659599218,1.0,7,5,37916874.0,292.0,NaN,NaN,None,0d343cc4d29c5395b07b14aba34c8961,NaN,2bd656317f119ec3bc2564cf1bfbcd4a,0d343cc4d29c5395b07b14aba34c8961,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444249,458622,3ba8c1e455e27172edec64deff3a8b15,1662339887,2.0,7,16,44233.0,202.0,NaN,NaN,None,3ca952f8c46a684dd61be10b51ee6546,NaN,0025132fbe92679e3472ba869b60af35,c0cbca2bf0f53ca325b6661f245db4fd,05b18ab3453ed98d5fcb899336e248a3
11444250,458622,3ba8c1e455e27172edec64deff3a8b15,1662340090,2.0,7,17,44233.0,202.0,NaN,NaN,None,3ca952f8c46a684dd61be10b51ee6546,NaN,0025132fbe92679e3472ba869b60af35,c0cbca2bf0f53ca325b6661f245db4fd,05b18ab3453ed98d5fcb899336e248a3
11444251,458622,3ba8c1e455e27172edec64deff3a8b15,1662340294,2.0,7,18,44233.0,202.0,NaN,NaN,None,3ca952f8c46a684dd61be10b51ee6546,NaN,0025132fbe92679e3472ba869b60af35,c0cbca2bf0f53ca325b6661f245db4fd,05b18ab3453ed98d5fcb899336e248a3
11444252,458622,3ba8c1e455e27172edec64deff3a8b15,1662340497,2.0,7,19,44233.0,202.0,NaN,NaN,None,3ca952f8c46a684dd61be10b51ee6546,NaN,0025132fbe92679e3472ba869b60af35,c0cbca2bf0f53ca325b6661f245db4fd,05b18ab3453ed98d5fcb899336e248a3


In [81]:
count_genre = train_data['genre_id'].value_counts()

# Convert the result to a DataFrame
song_id_counts_df = count_genre.reset_index()

# Rename the columns for clarity
song_id_counts_df.columns = ['song_id', 'count']

print(song_id_counts_df['song_id'][0])

b856b6781d370a3645c6dde0c20b3597


In [51]:
test_data = pd.merge(label_test_dropped, meta_song, on='song_id', how='left')
test_data

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,598,43519381e8351d7c6afcc66bcc0a0636,1660177283,1.0,7,1,49372.0,219.0,842733.0,62.0,2000-01,NaN,b856b6781d370a3645c6dde0c20b3597,5b39a88bd5d9e544fd791f85faadb0fc,NaN,3ddf6f1cfae822d8211d6c69ce90fcec
1,598,928e2e8e316f29efd986fa8d23429431,1660177503,1.0,7,2,29463.0,234.0,161225.0,62.0,2010-11,06e0e72f84f34863e4a69d90674f0af5,bb737ea7450c3abbab1ff613a2c6309e,5c544fc795be3b958137b0c1288162ac,NaN,a8c81c17c0b86c059edddfa4d7c1a703
2,598,5229859624d3b58c35646154cf366bf4,1660177582,1.0,7,3,395.0,231.0,309111.0,52.0,2011-01,NaN,b856b6781d370a3645c6dde0c20b3597,5c69a60ce3271a7ba885d88ca4810ccd,NaN,54a6076fa42c3959d2aae01d3a55df6d
3,598,49e50187f7a7f658623807780f499a15,1660177584,1.0,7,4,10939.0,216.0,3931773.0,62.0,2010-09,NaN,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,NaN,845efbc9f4e9cbd6603ba2a55092696d
4,598,bc8b660fe4d4bfa0a13334beae0cb750,1660177587,1.0,7,5,31927.0,193.0,NaN,NaN,None,NaN,NaN,c1079ef109db2aba72f78c632ab73803,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860967,714498,13d62012feb791ded330b17098230e0d,1666147178,0.0,7,16,41290862.0,83.0,185582850.0,52.0,2022-07,NaN,6ea61b86b8fff0e3a05bc73ea4eaf21f,8b5631323f7123297766b5241b759168,NaN,NaN
2860968,714498,a3f4cb9a06f48f2e9ea381be3afc64b3,1666147262,0.0,7,17,41290862.0,103.0,185582850.0,52.0,2022-07,NaN,65aeeca3341ca1c6a2ed774aa4e22add,8b5631323f7123297766b5241b759168,NaN,NaN
2860969,714498,9bb5e3b8b43fe1e306459c89ab263823,1666147366,0.0,7,18,41290862.0,98.0,185582850.0,52.0,2022-07,NaN,65aeeca3341ca1c6a2ed774aa4e22add,8b5631323f7123297766b5241b759168,NaN,NaN
2860970,714498,e1ec04cca3d78e9e7fd9793ae45d3638,1666147464,0.0,7,19,41290862.0,92.0,185582850.0,52.0,2022-07,NaN,6ea61b86b8fff0e3a05bc73ea4eaf21f,8b5631323f7123297766b5241b759168,NaN,NaN


In [56]:
test_data.head(20)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,598,43519381e8351d7c6afcc66bcc0a0636,1660177283,1.0,7,1,49372.0,219.0,842733.0,62.0,2000-01,NaN,b856b6781d370a3645c6dde0c20b3597,5b39a88bd5d9e544fd791f85faadb0fc,NaN,3ddf6f1cfae822d8211d6c69ce90fcec
1,598,928e2e8e316f29efd986fa8d23429431,1660177503,1.0,7,2,29463.0,234.0,161225.0,62.0,2010-11,06e0e72f84f34863e4a69d90674f0af5,bb737ea7450c3abbab1ff613a2c6309e,5c544fc795be3b958137b0c1288162ac,NaN,a8c81c17c0b86c059edddfa4d7c1a703
2,598,5229859624d3b58c35646154cf366bf4,1660177582,1.0,7,3,395.0,231.0,309111.0,52.0,2011-01,NaN,b856b6781d370a3645c6dde0c20b3597,5c69a60ce3271a7ba885d88ca4810ccd,NaN,54a6076fa42c3959d2aae01d3a55df6d
3,598,49e50187f7a7f658623807780f499a15,1660177584,1.0,7,4,10939.0,216.0,3931773.0,62.0,2010-09,NaN,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,NaN,845efbc9f4e9cbd6603ba2a55092696d
4,598,bc8b660fe4d4bfa0a13334beae0cb750,1660177587,1.0,7,5,31927.0,193.0,NaN,NaN,None,NaN,NaN,c1079ef109db2aba72f78c632ab73803,NaN,NaN
5,598,743724b23a44300df22d204cf0f3702e,1660177588,1.0,7,6,11875.0,270.0,160419.0,62.0,2010-11,NaN,a2a4a0943fe2c5fe6891d9c34dca906f,9cc814abf8f2edfe616f36934986c81b,NaN,ef7f4408fc11bbb0a715f33184fc2e60
6,598,12646130ef79f2d8fb7796e1586f58c4,1660177589,1.0,7,7,56624.0,247.0,167873.0,62.0,2010-07,25caf9b76193b150d3b34bcf3b9f5bc1,a2a4a0943fe2c5fe6891d9c34dca906f,9cc814abf8f2edfe616f36934986c81b,NaN,f0ef0ec96b11f6da9f825bf68e7904d4
7,598,8a15b5b07ccef816853256013a6adc2d,1660177590,1.0,7,8,9420.0,231.0,186163.0,62.0,2010-09,NaN,a2a4a0943fe2c5fe6891d9c34dca906f,9cc814abf8f2edfe616f36934986c81b,NaN,1a1bbc0809be3a9e59e4e28a4fdfaba2
8,598,24d59f60232ddd07628179672ee007a1,1660177593,1.0,7,9,30734.0,252.0,156470.0,62.0,2010-09,1c953a7677f9b65153930945f00e91f8,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,1c953a7677f9b65153930945f00e91f8,155cce61af78087713ce794ceee57fb3
9,598,b75bcf271a082f0141c1ab0acfd86c43,1660177594,1.0,7,10,31325.0,168.0,149394.0,62.0,2010-01,NaN,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,NaN,a430763e8c60fc0b5b61bd74cd3489a0


In [53]:
test2 = test_data.dropna()

In [54]:
test2

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
8,598,24d59f60232ddd07628179672ee007a1,1660177593,1.0,7,9,30734.0,252.0,156470.0,62.0,2010-09,1c953a7677f9b65153930945f00e91f8,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,1c953a7677f9b65153930945f00e91f8,155cce61af78087713ce794ceee57fb3
15,598,a689c32ea9ae6bb905817aed73df6159,1660177604,1.0,7,16,23999.0,255.0,31207671.0,62.0,2010-08,15629de3f2a8112911cc5b9e90b300be,a2a4a0943fe2c5fe6891d9c34dca906f,de482ba3ba9c5cc61becdaae358ecc78,15629de3f2a8112911cc5b9e90b300be,3bd27ed5baab570150337b1b2edd70f2
25,1039,e23d3935f4e00f40a906815d46e317b3,1660182706,1.0,7,6,184566.0,316.0,57554709.0,24.0,2019-07,4baf5ef7e97acd974e5ab0872ec9c0d8,b856b6781d370a3645c6dde0c20b3597,4d7da29a158569094e1f71a4ade88f6b,e15df56d991695ad0bed5a676f894ff9,ac95cc788a838fb7813eda942c19fc66
26,1039,247063e71d28e08010c4f4fa8301985a,1660182733,1.0,7,7,184566.0,283.0,105720727.0,24.0,2021-02,9cfe904b05005e0c1a05c4af254f40b9,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,dc2061c7d3853083596b0ce24cb806f2,4531dafc96736d3af7efb59aa0bb5bf9
36,1039,8bbb6c351258ed242f4907beff28b0d5,1660183382,1.0,7,17,13755.0,198.0,42149006.0,24.0,2018-11,f53d71dc91b08f01b7dd0edd995fbf56,bb3d7b04b67d5aeb5ab145bdd70750da,bafe4ea3c99161d13843b345569a6d1a,0f612401f31a00017eca62ffaeb1a187,f53d71dc91b08f01b7dd0edd995fbf56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860919,712209,b60bab9e0f2350342a0ef96b9aa27676,1666257354,2.0,7,8,31573048.0,173.0,177940437.0,3.0,2022-04,c714e08e3c4574cca3a91496b590bd40,ce4db56f6a48426643b08038139a8a75,2b8913b897b53432a57fddb390777b74,d4959a859178bb8d8b9e3b2ced41a710,480d2652e201470ec5990a2959779fb7
2860923,712209,379cce803f6e6f1883057fd39c0d08cb,1666257927,2.0,7,12,46457.0,189.0,765549.0,62.0,2012-05,c2828a52f267118f923b941b82817872,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,dc27b9eedec5bdaba19277a35de862d4,c2828a52f267118f923b941b82817872
2860925,712209,1d52ee1b8be269a14a7810a6ff3296d2,1666258259,2.0,7,14,12719146.0,175.0,51496821.0,3.0,2019-04,4cb96a84bd5f2f6701b6258219168565,ce4db56f6a48426643b08038139a8a75,f9b7f48dbd07a9979e64ccf88af181aa,a23f78784291a8a1961b94c74af71d6f,4cb96a84bd5f2f6701b6258219168565
2860927,712209,b60bab9e0f2350342a0ef96b9aa27676,1666307874,2.0,7,16,31573048.0,173.0,177940437.0,3.0,2022-04,c714e08e3c4574cca3a91496b590bd40,ce4db56f6a48426643b08038139a8a75,2b8913b897b53432a57fddb390777b74,d4959a859178bb8d8b9e3b2ced41a710,480d2652e201470ec5990a2959779fb7
